In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""


@author: Deepak
"""
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os.path
from keras.utils import np_utils
import tensorflow as tf
from pandas import Series
import time
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam


checkpointer = ModelCheckpoint(
        filepath='/home/cis/Desktop/LStm Dense Trajectories/data/checkpoints/' + 'bow'+ \
            '.{epoch:03d}-{val_loss:.3f}.hdf5',
        verbose=1,
        save_best_only=True)

# Helper: TensorBoard
tb = TensorBoard(log_dir='/home/cis/Desktop/LStm Dense Trajectories/data/logs')

# Helper: Stop when we stop learning.
early_stopper = EarlyStopping(patience=10)

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger('/home/cis/Desktop/LStm Dense Trajectories/data/logs/'+ '-' + 'training-' + \
        str(timestamp) + '.log')

Using TensorFlow backend.


In [2]:
trainingdict = loadmat('/home/cis/Videos/DSC550/SVM/All desc/dictan_Alldesc1.mat')
train_x = trainingdict['encode']
train_y = trainingdict['label']


testing_dict = loadmat('/home/cis/Videos/DSC550/SVM/All desc/dictan_Alldesc1_t.mat')
test_x = testing_dict['encode_t']
test_y = testing_dict['label_t']

classes = ['boxing' , 'handclapping' , 'handwaving' , 'jogging' , 'running' , 'walking']


In [3]:
## Convert label to categorical
def convert_to_categ(train_y):
    y = []
    for i in range(0 , len(train_y)):
        k= train_y[i][0]
        #print k
        label_encoded = classes.index(k)
        get_label = np_utils.to_categorical (label_encoded, len(classes))       
        get_label  = get_label[0]
        y.append(get_label)
        
    return np.array(y)



train_x_t = train_x.reshape(train_x.shape[0], 1, train_x.shape[1])
test_x_t = test_x.reshape(test_x.shape[0], 1, test_x.shape[1])    
 
def convert_to_categ_test(test_y):
    y = []
    for i in range(0 , len(test_y)):
        k= test_y[i][0]
        #print k
        label_encoded = classes.index(k)
        get_label = np_utils.to_categorical (label_encoded, len(classes))       
        get_label  = get_label[0]
        y.append(get_label)
        
    return np.array(y)

In [4]:
train_y = convert_to_categ(train_y)
test_y = convert_to_categ_test(test_y)

In [5]:
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(train_x_t.shape[1], train_x_t.shape[2]),dropout=0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

optimizer = Adam(lr=1e-6)
model.compile(loss='categorical_crossentropy', optimizer= optimizer,
                           metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 200)            87405600  
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 606       
Total params: 87,426,306
Trainable params: 87,426,306
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(train_x_t, train_y, batch_size=32, epochs=70, validation_data=(test_x_t, test_y), verbose=1, callbacks=[checkpointer, tb, early_stopper, csv_logger])

Train on 335 samples, validate on 264 samples
Epoch 1/70
335/335 [==============================] - 91s - loss: 1.7984 - acc: 0.1642 - val_loss: 1.7873 - val_acc: 0.2159
Epoch 2/70
335/335 [==============================] - 90s - loss: 1.7918 - acc: 0.1791 - val_loss: 1.7820 - val_acc: 0.2689
Epoch 3/70
335/335 [==============================] - 109s - loss: 1.7824 - acc: 0.2269 - val_loss: 1.7766 - val_acc: 0.3447
Epoch 4/70
335/335 [==============================] - 67s - loss: 1.7705 - acc: 0.2776 - val_loss: 1.7713 - val_acc: 0.4091
Epoch 5/70
335/335 [==============================] - 71s - loss: 1.7745 - acc: 0.2985 - val_loss: 1.7661 - val_acc: 0.4659
Epoch 6/70
335/335 [==============================] - 92s - loss: 1.7627 - acc: 0.2985 - val_loss: 1.7608 - val_acc: 0.5076
Epoch 7/70
335/335 [==============================] - 65s - loss: 1.7600 - acc: 0.3493 - val_loss: 1.7555 - val_acc: 0.5492
Epoch 8/70
335/335 [==============================] - 79s - loss: 1.7514 - acc: 0.394

335/335 [==============================] - 55s - loss: 1.6554 - acc: 0.7463 - val_loss: 1.6550 - val_acc: 0.8674
Epoch 25/70
335/335 [==============================] - 65s - loss: 1.6437 - acc: 0.7134 - val_loss: 1.6485 - val_acc: 0.8788
Epoch 26/70
335/335 [==============================] - 58s - loss: 1.6315 - acc: 0.8060 - val_loss: 1.6418 - val_acc: 0.8826
Epoch 27/70
335/335 [==============================] - 67s - loss: 1.6281 - acc: 0.7881 - val_loss: 1.6352 - val_acc: 0.8826
Epoch 28/70
335/335 [==============================] - 71s - loss: 1.6191 - acc: 0.7761 - val_loss: 1.6284 - val_acc: 0.8826
Epoch 29/70
335/335 [==============================] - 87s - loss: 1.6114 - acc: 0.7881 - val_loss: 1.6217 - val_acc: 0.8788
Epoch 30/70
335/335 [==============================] - 62s - loss: 1.6057 - acc: 0.7910 - val_loss: 1.6148 - val_acc: 0.8788
Epoch 31/70
335/335 [==============================] - 114s - loss: 1.6001 - acc: 0.7791 - val_loss: 1.6080 - val_acc: 0.8788
Epoch 32/70

335/335 [==============================] - 65s - loss: 1.4843 - acc: 0.8866 - val_loss: 1.4955 - val_acc: 0.8864
Epoch 48/70
335/335 [==============================] - 74s - loss: 1.4720 - acc: 0.8925 - val_loss: 1.4882 - val_acc: 0.8864
Epoch 49/70
335/335 [==============================] - 74s - loss: 1.4660 - acc: 0.8836 - val_loss: 1.4810 - val_acc: 0.8788
Epoch 50/70
335/335 [==============================] - 78s - loss: 1.4562 - acc: 0.8776 - val_loss: 1.4738 - val_acc: 0.8826
Epoch 51/70
335/335 [==============================] - 71s - loss: 1.4437 - acc: 0.9134 - val_loss: 1.4665 - val_acc: 0.8826
Epoch 52/70
335/335 [==============================] - 74s - loss: 1.4358 - acc: 0.9015 - val_loss: 1.4591 - val_acc: 0.8826
Epoch 53/70
335/335 [==============================] - 73s - loss: 1.4355 - acc: 0.9045 - val_loss: 1.4516 - val_acc: 0.8939
Epoch 54/70
335/335 [==============================] - 91s - loss: 1.4146 - acc: 0.9284 - val_loss: 1.4444 - val_acc: 0.8977
Epoch 55/70


335/335 [==============================] - 74s - loss: 1.3075 - acc: 0.9313 - val_loss: 1.3296 - val_acc: 0.9129
